# Checkpoint 2 Notebook

Descriptions of columns in dataset: https://data.lacity.org/Public-Safety/Crime-Data-from-2020-to-Present/2nrs-mtv8/about_data

(5 points) Data preprocessing: (a) import, (b) parse (e.g., convert strings to ints), (c)
organize (e.g., set up a database or a pandas DataFrame).

(20 points) Basic data exploration and summary statistics
○ You must present three conclusions using at least three different statistical
methods including hypothesis testing.
■ For example: What are the main characteristics of your dataset? How
many features and entries are there? Is a feature over-represented? Are
features correlated? Are there outliers? Identify the attributes that will
affect your choice of primary analysis technique. Etcetera.
○ For each method, you must have at least one gorgeous plot.

## Data Preprocessing

Since our dataset is so large, we had to split it into multiple files to be able to upload it to GitHub. Therefore, the first step in our data preprocessing is to import all the files and concatenate them into a single DataFrame. We will use the pandas library for this task.


In [1]:
import pandas as pd


def load_full_crime_data_df():
    year_range = list(range(2020, 2026))
    main_df = None
    for year in year_range:
        df = pd.read_csv(f"data/crime_data_{year}.csv")
        if main_df is None:
            main_df = df
        else:
            main_df = pd.concat([main_df, df], ignore_index=True)
    return main_df


In [2]:
df = load_full_crime_data_df()

In [3]:
df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON,Year
0,211507896,04/11/2021 12:00:00 AM,2020-11-07,845,15,N Hollywood,1502,2,354,THEFT OF IDENTITY,...,Invest Cont,354.0,NaN,NaN,NaN,7800 BEEMAN AV,NaN,34.2124,-118.4092,2020
1,201516622,10/21/2020 12:00:00 AM,2020-10-18,1845,15,N Hollywood,1521,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",...,Invest Cont,230.0,NaN,NaN,NaN,ATOLL AV,N GAULT,34.1993,-118.4203,2020
2,240913563,12/10/2024 12:00:00 AM,2020-10-30,1240,9,Van Nuys,933,2,354,THEFT OF IDENTITY,...,Invest Cont,354.0,NaN,NaN,NaN,14600 SYLVAN ST,NaN,34.1847,-118.4509,2020
3,210704711,12/24/2020 12:00:00 AM,2020-12-24,1310,7,Wilshire,782,1,331,THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND ...,...,Invest Cont,331.0,NaN,NaN,NaN,6000 COMEY AV,NaN,34.0339,-118.3747,2020
4,201418201,10/03/2020 12:00:00 AM,2020-09-29,1830,14,Pacific,1454,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),...,Invest Cont,420.0,NaN,NaN,NaN,4700 LA VILLA MARINA,NaN,33.9813,-118.4350,2020


In [4]:
df.isnull().sum()

DR_NO                   0
Date Rptd               0
DATE OCC                0
TIME OCC                0
AREA                    0
AREA NAME               0
Rpt Dist No             0
Part 1-2                0
Crm Cd                  0
Crm Cd Desc             0
Mocodes            151619
Vict Age                0
Vict Sex           144644
Vict Descent       144656
Premis Cd              16
Premis Desc           588
Weapon Used Cd     677744
Weapon Desc        677744
Status                  1
Status Desc             0
Crm Cd 1               11
Crm Cd 2           935831
Crm Cd 3          1002677
Crm Cd 4          1004927
LOCATION                0
Cross Street       850755
LAT                     0
LON                     0
Year                    0
dtype: int64

We will convert the "Date Rptd" column to datetime format for easier analysis later on.

In [5]:
df["Date Rptd"] = pd.to_datetime(
    df["Date Rptd"], format="%m/%d/%Y %I:%M:%S %p", errors="coerce"
)

Additionally we will combine the "DATE OCC" and "TIME OCC" columns into a single datetime column called "Date Occurred".

In [6]:
df["DATE OCC"] = pd.to_datetime(df["DATE OCC"], errors="coerce")

time_str = df["TIME OCC"].fillna(0).astype(int).astype(str).str.zfill(4)

date_part = df["DATE OCC"].dt.strftime("%Y-%m-%d")
combined = (
    date_part + " " + time_str.str.slice(0, 2) + ":" + time_str.str.slice(2, 4) + ":00"
)
df["DATE OCC"] = pd.to_datetime(combined, format="%Y-%m-%d %H:%M:%S", errors="coerce")
df = df.drop(columns=["TIME OCC"])

In [7]:
for col in df.columns:
    print(
        f"Column '{col}' has {df[col].isnull().sum()} missing values. Column dtype: {df[col].dtype}"
    )

Column 'DR_NO' has 0 missing values. Column dtype: int64
Column 'Date Rptd' has 0 missing values. Column dtype: datetime64[ns]
Column 'DATE OCC' has 0 missing values. Column dtype: datetime64[ns]
Column 'AREA' has 0 missing values. Column dtype: int64
Column 'AREA NAME' has 0 missing values. Column dtype: object
Column 'Rpt Dist No' has 0 missing values. Column dtype: int64
Column 'Part 1-2' has 0 missing values. Column dtype: int64
Column 'Crm Cd' has 0 missing values. Column dtype: int64
Column 'Crm Cd Desc' has 0 missing values. Column dtype: object
Column 'Mocodes' has 151619 missing values. Column dtype: object
Column 'Vict Age' has 0 missing values. Column dtype: int64
Column 'Vict Sex' has 144644 missing values. Column dtype: object
Column 'Vict Descent' has 144656 missing values. Column dtype: object
Column 'Premis Cd' has 16 missing values. Column dtype: float64
Column 'Premis Desc' has 588 missing values. Column dtype: object
Column 'Weapon Used Cd' has 677744 missing values.

Mocodes stands for Modus Operandi Codes, which are codes used to describe the method or manner in which a crime was committed. We will fill any missing values in this column with "Unknown".

In [8]:
df["Mocodes"] = df["Mocodes"].fillna("Unknown")

For the "Vict Sex" and "Vict Descent" columns, we have filled any missing values with "X" to indicate that the information is not available, which is the convention used in the dataset.

In [9]:
df["Vict Sex"] = df["Vict Sex"].fillna("X")
df["Vict Descent"] = df["Vict Descent"].fillna("X")

In [10]:
df["Premis Cd"] = df["Premis Cd"].fillna(-1.0)
df["Premis Desc"] = df["Premis Desc"].fillna("Unknown")

In [11]:
df["Weapon Used Cd"] = df["Weapon Used Cd"].fillna(-1.0)
df["Weapon Desc"] = df["Weapon Desc"].fillna("Unkown")

In [12]:
df["Status"] = df["Status"].fillna("Unknown")

In [13]:
df["Crm Cd 1"] = df["Crm Cd 1"].fillna(-1.0)
df["Crm Cd 2"] = df["Crm Cd 2"].fillna(-1.0)
df["Crm Cd 3"] = df["Crm Cd 3"].fillna(-1.0)
df["Crm Cd 4"] = df["Crm Cd 4"].fillna(-1.0)

In [14]:
df["Cross Street"] = df["Cross Street"].fillna("N/A")